<a href="https://colab.research.google.com/github/joelm67/Remote-Sensing-Python/blob/main/Copy_of_introduction_to_gdal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing GDAL

Now that we learned about some of the basic concepts of Python as well as importaing libraries and arrays, let's dive in and start doing image processing.  For this part we are going to do several things.  We are going to import the GDAL library so that we can use it to read and manipulate remotely sensed imagery in geospatial context.  Second we are going to use Numpy capabilities to calculate NDVI and write the file back to the disk using GDAL that can be downloaded.

First, lets learn a bit more about importing libraries.  So far, we imported libraries using a simple *import xxxxxx from xx* syntax.  This works because the libraries (Numpy, matplotlib etc) we used so far are common libraries that our Python language knows how to find.  However, the GDAL library we will now import is more *exotic* - meaning that we need to tell our Python interpreter how to find it.  This is done by interacting with the operating system (OS)! 

The notebook we are working in is a Jupyter notebook that lives on top of a linux virtual machine.  You can think of it as a computer in the cloud with a linux operating system.  The first thing we need to do is to check if the operating system has any updates - just like how you would check for updates for your smart phone.  The `!apt update` and the `!apt upgrade` does that. Next, we are going to install gdal tools.  The line including `!apt install gdal-bin python-gdal python3-gdal` does that.  Note that to interact with the OS, we are using the `!` character (escape character) to escape to the OS! Finally, now that our Python interpreter knows how to find GDAL, we can import it as we import other common libraries.  **Warning: this could take a bit!**

In [ ]:
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
import gdal
print(gdal.VersionInfo())
import matplotlib.pyplot as plt

# Using GDAL

Now that we have made our Python code GDAL capabale, let's start using it.  We are going to use gdal to import our data, read other attributes about the data, and write some results back to an image file.  Here are the steps:
- copy image data from a public server in the cloud to local OS
- open that file using `gdal.Open`
- read data from the opened file using `GetRasterBand` and `ReadAsArray` functions
- print some information about the array (image)
- calculate NDVI
- write the NDVI file back to the OS using `gdal.xxxx`

In [ ]:
# grab the image data from the cloud using wget and copy to a local file called "q3.tif"
!wget "https://storage.googleapis.com/alexi_daily/EnvSt956/LS_228078_2005q3.tif" -O q3.tif
# open the local file using gdal open into a variable called ds
ds = gdal.Open("q3.tif", gdal.GA_ReadOnly)
# lets grab some information about this dataset
print("Driver: {}/{}".format(ds.GetDriver().ShortName,
                            ds.GetDriver().LongName))
print("Size is {} x {} x {}".format(ds.RasterXSize,
                                    ds.RasterYSize,
                                    ds.RasterCount))
print("Projection is {}".format(ds.GetProjection()))
geotransform = ds.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

!gdalinfo q3.tif

Let's break this down.  `gdal.Open` as interpreted is using the `Open` method that belongs to `gdal`, which comes from importing the gdal library.  The `GA_ReadOnly` method opens the file for reading only i.e. it will only allow you to read from the file but not write to it (potentially to prevent accidental overwrite of the file).  Of course when you want to generate a new file and write to the disk, you will want to open the new file with Write permission.

Now that the file is opened, we can grab lots of information about that file.  For example you may want to know its dimensions, or projection, or pixel size and etc.
- `GetDriver()` returns the file format
- `RasterXSize` returns X dimension
- `RasterYSize` returns Y dimension
- `RasterCount` returns number of bands
- `GetProjection()` returns geodetic projection
- `GetGeoTransform()` returns projection transformation parameters with:

  `GeoTransform[0]` is top left X coordinate

  `GeoTransform[1]` w-e pixel resolution

  `GeoTransform[2]` 0

  `GeoTransform[3]` top left y coordinate

  `GeoTransform[4]` 0

  `GeoTransform[5]` n-s pixel resolution (negative value)


---




Next is to fetch a raster band, which is done one band at a time for now.  Note that fetching data about the band into an object and reading the band data into an array are different things!  What we are doing below is to fetch band as an object but not as a data array.

In [ ]:
# read band 1 from the dataset into a variable (object) called band
# note 1-based counting
# band1 = blue
# band2 = green
# band3 = red
# band4 = NIR
# band5 = SWIR1
# band6 = SWIR2
band = ds.GetRasterBand(5) # get band 1
# print data type of the band 
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

# grab the minimum and maximum values of the band
min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))

if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))

if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))


Next, we'll read data using array option.

In [ ]:
import numpy as np
# read the first band of the image into an array (NOTE 1-based counting)
# it is read as a numpy array
b1 = ds.GetRasterBand(1).ReadAsArray() # band 1
# print image details (dimensions)
print('shape of the array is:')
print(b1.shape)
print('the minimumm value of the image is:')
print(b1.min())
print('the maximum value of the image is:')
print(b1.max())
#plt.imshow(b1, vmin = 0, vmax = 2000, interpolation='nearest', aspect='auto')
#plt.colorbar()
#plt.show()

# let's do NDVI example
# (nir-red)/(nir+red)
red = ds.GetRasterBand(3).ReadAsArray() # red band
nir = ds.GetRasterBand(4).ReadAsArray() # NIR band
ndvi = (nir - red)/(nir + red)
print(np.sum(ndvi))
print('old min = ',ndvi.min())
print('old max = ',ndvi.max())
ndvi = np.where(ndvi > 1.0, 1.0, ndvi)
print("\n")
print('new min = ',ndvi.min())
print('new max = ',ndvi.max())
plt.imshow(ndvi, vmin = 0, vmax = 1, interpolation='nearest', aspect='auto')
plt.colorbar()
plt.show()

Now that we have an NDVI image, let's write it to a new image.  The file writing (file creation) process is a five step process:
- select a driver (driver means file type). Here is a list of drivers supported by GDAL [GDAL drivers](https://gdal.org/drivers/raster/index.html)
- create an empty file (shell) with that driver and correct dimensions
- populate the metadata properties of the output file
- populate the pixels with an array
- write the file to the disk

In [ ]:
# grab dimensions of the file
[cols, rows] = ndvi.shape
# use the Geotiff driver
driver = gdal.GetDriverByName("GTiff")
# create an output file data object
outdata = driver.Create('myndvi.tif', rows, cols, 1, gdal.GDT_Float32,[ 'COMPRESS=LZW' ])
outdata.SetGeoTransform(ds.GetGeoTransform()) # sets same geotransform as input
outdata.SetProjection(ds.GetProjection()) # sets same projection as input
outdata.GetRasterBand(1).WriteArray(ndvi)
#outdata.GetRasterBand(1).SetNoDataValue(-99.0) # if you want these values transparent
outdata.FlushCache() # saves to disk!!
outdata = None # closes the file
# you can check if the file exist and has the correct dimensions/type
!gdalinfo myndvi.tif

Finally, we can download this disk file to your local computer using a colab library

In [ ]:
from google.colab import files
files.download('myndvi.tif')